# Monitoring de la VFC

1.	Évolution de la FC (fréquence cardiaque) :
- Comparez la moyenne de la fréquence cardiaque pour chaque jour/semaine.
- Représentez cela dans un graphique (évolution journalière ou hebdomadaire).

2.	Analyses temporelles et spectrales :
- Temporelles : Moyenne RR, SDNN (écart-type des intervalles RR), RMSSD.
- Spectrales : Low Frequency (LF), High Frequency (HF), ratio LF/HF, Welch, FFT
